In [ ]:
from openai import OpenAI
import pandas as pd
import base64
import os
import re

client = OpenAI(
    api_key="",
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

imgs_questions = pd.read_csv("../Dataset/SoMDataset/Relational/3_color_size_shape/3_color_size_shape.csv")

# 获取所有描述文本（假设文本在第二列）
text_list = imgs_questions.iloc[18:, 1]  # 获取文本列

# 本地图片文件夹路径
folder_path = "../Dataset/SoMDataset/Relational/3_color_size_shape/imgs" 

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

def create_message(image_path, text):
    return [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": text+",不使用序号，使用(0,0)到(10,10)内的坐标表示"
        },
        {
          "type": "image_url",
          "image_url": {
            "url":  f"data:image/jpeg;base64,{encode_image(image_path)}"
          },
        },
      ],
    }
  ]

# 获取文件夹中的所有图片
image_files = sorted(
    [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png'))],
    key=lambda x: int(re.match(r'(\d+)', x).group(1))
)

# 遍历文件夹中的所有图片文件和文本
for idx, filename in enumerate(image_files):
    image_path = os.path.join(folder_path, filename)
    text = text_list.iloc[idx]  # 从 text_list 获取当前图片对应的文本
    response = client.chat.completions.create(
    model="gemini-1.5-flash",
    messages=create_message(image_path,text)
    )

    print(response.choices[0])